In [40]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# CUSTOMER PURCHASE PATTERN

In [41]:
cust_txn = pd.read_excel("data/Customer_Transaction.xlsx")

In [42]:
cust_txn.head(2)

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,1,3,-169.0,0.0,-3
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,1,1,0.0,0.0,0


In [43]:
len(cust_txn)

544649

In [44]:
cust_txn.City_Name.unique()

array(['Dubai', nan, 'Abu Dhabi', 'Ajman', 'Al Ain', 'Fujairah',
       'Ras Al Khaimah', 'Sharjah'], dtype=object)

In [45]:
cust_txn.Transaction_Type.unique()

array(['Return', 'Purchase'], dtype=object)

In [46]:
cust_txn.Week.max()

53

In [47]:
# dropping columns which has no distinct values or not useful
cust_txn.drop(['Territory', 'Business', 'Units_Sold', 'Transaction_Type', 'City_Name', 'Store_Type'],\
              axis=1, inplace=True)

In [48]:
cust_group = cust_txn.groupby(['Customer_ID']).sum()

In [49]:
cust_txn.Return_Reason.fillna('None', inplace=True)

In [50]:
def  check_return_reason(x):
    for char in x:
        if char != 'None':
            return 1
    return 0

In [51]:
cust_group[['Last_Pur_Year', 'Return_Reason']] = cust_txn.groupby(['Customer_ID']).\
                                            agg({'Year': lambda x: 2018 - max(x), 
                                                   'Return_Reason': check_return_reason})

In [52]:
cust_group.reset_index(inplace=True)

In [53]:
cust_group.drop("Store_Code", inplace=True, axis=1)

In [54]:
cust_group.head()

,Customer_ID,Year,Week,Invoices,Item_Count,Revenue,Discount,Last_Pur_Year,Return_Reason
0,1800000006365760,8066,110,5,38,1117.00,56.0,1,0
1,1800000006365820,2016,21,1,6,260.00,92.0,2,0
2,1800000006366060,12096,135,7,23,607.00,140.0,2,0
3,1800000006366230,12099,199,6,23,643.80,66.2,1,0
4,1800000006366570,20163,258,11,45,1285.01,90.0,1,1


In [55]:
cust_group.drop(['Year'], inplace=True, axis=1)

In [56]:
cust_group.Week.describe()

count    100000.000000
mean        146.376210
std         184.881066
min           0.000000
25%          39.000000
50%          83.000000
75%         183.000000
max        3350.000000
Name: Week, dtype: float64

In [57]:
cust_group.loc[cust_group.Invoices==1, "Invoices"] = 0
cust_group.loc[np.logical_and(cust_group.Invoices > 1,  cust_group.Invoices < 3), "Invoices"] = 1
cust_group.loc[np.logical_and(cust_group.Invoices >= 3,  cust_group.Invoices < 8), "Invoices"] = 2
cust_group.loc[cust_group.Invoices >= 8, "Invoices"] = 3

In [58]:
cust_group.loc[np.logical_and(cust_group.Item_Count >= 1,  cust_group.Item_Count < 5), "Item_Count"] = 0
cust_group.loc[np.logical_and(cust_group.Item_Count >= 5,  cust_group.Item_Count < 14), "Item_Count"] = 1
cust_group.loc[np.logical_and(cust_group.Item_Count >= 14,  cust_group.Item_Count < 35), "Item_Count"] = 2
cust_group.loc[cust_group.Item_Count >= 35, "Item_Count"] = 3

In [59]:
cust_group.loc[cust_group.Revenue < 130, "Revenue"] = 0
cust_group.loc[np.logical_and(cust_group.Revenue >= 130,  cust_group.Item_Count < 349), "Revenue"] = 1
cust_group.loc[np.logical_and(cust_group.Revenue >= 349,  cust_group.Item_Count < 881), "Revenue"] = 2
cust_group.loc[cust_group.Revenue >= 881, "Revenue"] = 3

In [60]:
cust_group.loc[cust_group.Discount < 4, "Discount"] = 0
cust_group.loc[np.logical_and(cust_group.Discount >= 4,  cust_group.Discount < 39), "Discount"] = 1
cust_group.loc[np.logical_and(cust_group.Discount >= 39,  cust_group.Discount < 145), "Discount"] = 2
cust_group.loc[cust_group.Discount >= 145, "Discount"] = 3

In [61]:
cust_group.loc[cust_group.Discount < 39, "Week"] = 0
cust_group.loc[np.logical_and(cust_group.Week >= 39,  cust_group.Discount < 83), "Week"] = 1
cust_group.loc[np.logical_and(cust_group.Week >= 83,  cust_group.Discount < 183), "Week"] = 2
cust_group.loc[cust_group.Week >= 183, "Week"] = 3

In [62]:
cust_group.describe()

,Customer_ID,Week,Invoices,Item_Count,Revenue,Discount,Last_Pur_Year,Return_Reason
count,1.000000e+05,100000.0,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.800000e+15,0.0,1.587130,1.540330,0.751450,1.520070,1.229770,0.198450
std,8.334842e+07,0.0,1.129877,1.098283,0.432174,1.103272,0.420687,0.398835
min,1.800000e+15,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.800000e+15,0.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,1.800000e+15,0.0,2.000000,2.000000,1.000000,2.000000,1.000000,0.000000
75%,1.800000e+15,0.0,3.000000,3.000000,1.000000,3.000000,1.000000,0.000000
max,1.800000e+15,0.0,3.000000,3.000000,1.000000,3.000000,2.000000,1.000000


# Find Customer - Store Code from transaction 

In [63]:
# cust_txn = pd.read_excel("data/Customer_Transaction.xlsx")

In [64]:
cust_txn.count()

Year             544649
Week             544649
Store_Code       544649
Return_Reason    544649
Customer_ID      544649
Invoices         544649
Item_Count       544649
Revenue          544649
Discount         544649
dtype: int64

In [65]:
cust_txn_group = cust_txn.groupby(['Customer_ID','Store_Code']).size().reset_index()

In [66]:
cust_txn_group.head()

,Customer_ID,Store_Code,0
0,1800000006365760,60067,4
1,1800000006365820,60054,1
2,1800000006366060,60066,1
3,1800000006366060,60069,1
4,1800000006366060,60079,2


In [67]:
cust_txn_group['Prediction'] = 1

In [68]:
cust_txn_group.count()

Customer_ID    184288
Store_Code     184288
0              184288
Prediction     184288
dtype: int64

In [69]:
# cust_txn_group.Store_Code.unique()

In [70]:
# prepare dataset for other combinations to 0
from itertools import product
Customer_ID = cust_txn_group.Customer_ID.unique()
Store_Code = cust_txn_group.Store_Code.unique()
cust_train = pd.DataFrame(list(product(Customer_ID, Store_Code)), columns=['Customer_ID', 'Store_Code'])

In [71]:
cust_train_merged = pd.merge(cust_train, cust_txn_group, how='left', left_on=['Customer_ID','Store_Code'], right_on=['Customer_ID','Store_Code'] )

In [72]:
cust_train_merged.head()

,Customer_ID,Store_Code,0,Prediction
0,1800000006365760,60067,4.0,1.0
1,1800000006365760,60054,NaN,NaN
2,1800000006365760,60066,NaN,NaN
3,1800000006365760,60069,NaN,NaN
4,1800000006365760,60079,NaN,NaN


In [73]:
cust_train_merged.fillna(0.0, inplace=True)

In [74]:
cust_train_merged.drop(0, axis=1, inplace=True)

In [75]:
cust_train_merged.drop(cust_train_merged[cust_train_merged.Store_Code == 60084].index, axis=0, inplace=True)

In [76]:
# sample zero entries to balance the dataset
# balanced_set = cust_train_merged[cust_train_merged.Prediction == 0.0]

In [77]:
# cust_train_merged = balanced_set.append(cust_train_merged[cust_train_merged.Prediction == 1.0])

In [78]:
cust_train_merged.Prediction.value_counts()

0.0    2915713
1.0     184287
Name: Prediction, dtype: int64

# EDA on Store

In [ ]:
store = pd.read_excel('data/Store_Master.xlsx')

In [ ]:
store.Geo_Field.fillna("UNKNOWN", inplace=True)

In [ ]:
store.tail()

In [ ]:
store["isMall"] = 1
store.loc[store["Mall_Name"] == "STANDALONE" ,"isMall"] = 0

In [ ]:
# dropping columns which has no distinct values or not useful
store.drop(['Territory', 'Business', 'Store_Name', 'Mall_Name', 'Train_Test_Store','Customer_Count', 'Total_Revenue',\
            'Store_Launch_Date', 'Region' ], inplace=True, axis=1)

In [ ]:
# store['Region'] = store.Region.map(lambda x: x.split('-')[1])

In [ ]:
store.Store_Format.fillna("Mall Stores", inplace=True)

In [ ]:
store.tail()

In [ ]:
# store_objects = store.select_dtypes(include=['object'])

In [ ]:
# store['Region']  = store['Region'].astype('category')
# store['Store_Format']  = store['Store_Format'].astype('category')

In [ ]:
# store['Region']  = store['Region'].cat.codes

In [ ]:
# store['Store_Format']  = store['Store_Format'].cat.codes

In [ ]:
store.isnull().sum()

In [ ]:
store.head()

# Demographics

In [ ]:
cust_demo = pd.read_excel("data/Customer_Demographics.xlsx")

In [ ]:
cust_demo.head()

In [ ]:
# removing columns which dont have distinct values or not relevant
cust_demo.drop(['Territory', 'Birth_date'], axis=1, inplace=True)

In [ ]:
cust_demo.Age.describe()

In [ ]:
cust_demo[cust_demo.Age<10]['Marital_Status'].head(5)

In [ ]:
cust_demo.isnull().sum()

In [ ]:
# converting dates to (current date - the date)days to see how active the user is
for i in ['First_txn_dt', 'Last_accr_txn_dt', 'Last_rdm_txn_dt']:
    cust_demo[i] = pd.to_datetime(cust_demo[i], format="%d%b%Y:%H:%M:%S")

In [ ]:
cust_demo.First_txn_dt.dtypes

In [ ]:
import datetime
NOW = datetime.datetime.now()
cust_demo['First_txn_days'] = cust_demo.First_txn_dt.map(lambda x: (NOW - x).days)
cust_demo['Last_accr_txn_days'] = cust_demo.Last_accr_txn_dt.map(lambda x: (NOW - x).days)
# cust_demo['Last_rdm_txn_days'] = cust_demo.Last_rdm_txn_dt.map(lambda x: (NOW - x).days)

In [ ]:
# removing columns which dont have distinct values or not relevant
cust_demo.drop(['First_txn_dt', 'Last_accr_txn_dt', 'Last_rdm_txn_dt'], axis=1, inplace=True)

In [ ]:
# fill Nan in dates to 9999

cust_demo.First_txn_days.fillna(99999, inplace=True)
cust_demo.Last_accr_txn_days.fillna(99999, inplace=True)
# cust_demo.Last_rdm_txn_days.fillna(99999, inplace=True)

In [ ]:
# reset null value in points to 0
cust_demo.Points.fillna(0, inplace=True)

In [ ]:
ids = cust_demo[cust_demo.Age==1]

In [ ]:
ids.head()

In [ ]:
# verified data with age == 1 has transactions in customer transaction table so we cant ignore them
pd.merge(ids, cust_txn, how='inner', left_on='Customer_ID', right_on='Customer_ID').head(5)

In [ ]:
cust_demo.Age.isnull().sum()/cust_demo.Age.count()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# plt.xlim(20, 60)
Agenan=cust_demo[~np.isnan(cust_demo.Age)]
sns.distplot(Agenan.Age)

In [ ]:
# removing columns which dont have distinct values or not relevant
cust_demo.drop(['Age'], axis=1, inplace=True)

In [ ]:
cust_demo.isnull().sum()

In [ ]:
cust_demo.head()

In [ ]:
cust_train_merged.count()

In [ ]:
cust_demo.to_csv("cust_demo.csv", index=False)

# MERGE and TRAIN SET

In [79]:
cust_demo = pd.read_csv("cust_df.csv")

In [80]:
store = pd.read_csv("store_df.csv")

In [81]:
store.head()

,Region_Code,Store_Code,Store_Format,Sales_Per_Day,Geo_Field,Store_Size_Sq_Ft,isMall
0,1,60077,0,0.0,24,0.0,1
1,5,60073,0,0.0,22,2.0,1
2,0,60065,0,3.0,12,2.0,1
3,0,60219,1,0.0,17,1.0,1
4,4,60071,0,2.0,26,2.0,0


In [82]:
train_df = pd.merge(cust_train_merged, cust_demo, how = 'left', left_on='Customer_ID', right_on='Customer_ID')

In [83]:
train_df.head(2)

,Customer_ID,Store_Code,Prediction,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,Points,First_txn_days,Last_accr_txn_days,Age
0,1800000006365760,60067,1.0,52,4,4,0,1,0,1,0,4,3,0,3
1,1800000006365760,60054,0.0,52,4,4,0,1,0,1,0,4,3,0,3


In [84]:
train_cus_df = pd.merge(train_df, cust_group, how = 'left', left_on='Customer_ID', right_on='Customer_ID')

In [85]:
# train_cus_df.to_csv("train_cus_df.csv", index=False)

In [3]:
# train_cus_df = pd.read_csv("train_cus_df.csv")

In [ ]:
# details of 60084 store is not in excel
# for i in train_df.Store_Code.unique():
#     for j in store.Store_Code.unique():
#         if i==j:
#             break
#         else:
#             pass
# print(i)

In [86]:
train_final_df = pd.merge(train_cus_df, store, how = 'left', left_on='Store_Code', right_on='Store_Code')

In [87]:
train_final_df.to_csv("final_train.csv", index=False)

In [88]:
len(train_final_df.columns)

28

In [ ]:
# index_drop = train_final_df[train_final_df.Store_Code == 60084].index

In [ ]:
# train_final_df.drop(index_drop, axis=0, inplace=True)

In [ ]:
# train_final_df.to_csv("train_df.csv", index=False)

# TEST SET 

In [89]:
import pandas as pd
test_df = pd.read_excel("data/Test_Set.xlsx")

In [90]:
test_df.count()

Customer_ID    400000
Store_Code     400000
dtype: int64

In [91]:
test_df_txn = pd.merge(test_df, cust_demo, how = 'left', left_on='Customer_ID', right_on='Customer_ID')

In [92]:
test_df_cus = pd.merge(test_df_txn,cust_group , how = 'left', left_on='Customer_ID', right_on='Customer_ID')

In [93]:
test_final_df = pd.merge(test_df_cus, store, how = 'left', left_on='Store_Code', right_on='Store_Code')

In [94]:
test_final_df.count()

Customer_ID           400000
Store_Code            400000
Nationality           400000
Income_Range          400000
Job_Type              400000
Marital_Status        400000
Gender                400000
State                 400000
Language              400000
Loyalty_Status        400000
Points                400000
First_txn_days        400000
Last_accr_txn_days    400000
Age                   400000
Week                  400000
Invoices              400000
Item_Count            400000
Revenue               400000
Discount              400000
Last_Pur_Year         400000
Return_Reason         400000
Region_Code           400000
Store_Format          400000
Sales_Per_Day         400000
Geo_Field             400000
Store_Size_Sq_Ft      400000
isMall                400000
dtype: int64

In [95]:
test_final_df.head()

,Customer_ID,Store_Code,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,...,Revenue,Discount,Last_Pur_Year,Return_Reason,Region_Code,Store_Format,Sales_Per_Day,Geo_Field,Store_Size_Sq_Ft,isMall
0,1800000006365760,60298,52,4,4,0,1,0,1,0,...,1.0,2.0,1,0,5,0,3.0,21,1.0,1
1,1800000006365760,60299,52,4,4,0,1,0,1,0,...,1.0,2.0,1,0,1,0,3.0,1,0.0,1
2,1800000006365760,60300,52,4,4,0,1,0,1,0,...,1.0,2.0,1,0,1,0,3.0,9,1.0,1
3,1800000006365760,60301,52,4,4,0,1,0,1,0,...,1.0,2.0,1,0,1,0,3.0,7,3.0,1
4,1800000006365820,60298,73,4,4,0,0,0,0,1,...,1.0,2.0,2,0,5,0,3.0,21,1.0,1


In [96]:
test_final_df.isnull().sum()

Customer_ID           0
Store_Code            0
Nationality           0
Income_Range          0
Job_Type              0
Marital_Status        0
Gender                0
State                 0
Language              0
Loyalty_Status        0
Points                0
First_txn_days        0
Last_accr_txn_days    0
Age                   0
Week                  0
Invoices              0
Item_Count            0
Revenue               0
Discount              0
Last_Pur_Year         0
Return_Reason         0
Region_Code           0
Store_Format          0
Sales_Per_Day         0
Geo_Field             0
Store_Size_Sq_Ft      0
isMall                0
dtype: int64

In [97]:
# details of 60084 store is not in excel. skipping that store
train_final_df.dropna(inplace=True)

In [99]:
test_final_df.to_csv("final_test.csv", index=False)

In [103]:
len(test_final_df.columns)

27

In [ ]:
train_final_df.to_csv("train_df_w.csv", index=False)
test_final_df.to_csv("test_df_w.csv", index=False)

# MODELLING

In [ ]:
ntrain = len(train_final_df)

In [ ]:
ntest = len(test_final_df)

In [ ]:
full_df = train_final_df.append(test_final_df)

In [ ]:
ntrain

In [ ]:
full_df.describe()

In [ ]:
full_df.loc[full_df.Nationality.isin(['UNDEFINED', 'Unspecified']),'Nationality'] = "UNKNOWN"

In [ ]:
full_df.loc[full_df.Income_Range.isin(['Unspecified', 'Unknown']),'Income_Range'] = "UNKNOWN"

In [ ]:
full_df.loc[full_df.Job_Type.isin(['Unspecified', 'Others']),'Job_Type'] = "UNKNOWN"

In [ ]:
full_df.loc[full_df.Marital_Status.isin(['Unspecified', 'Others']),'Marital_Status'] = "UNKNOWN"

In [ ]:
full_df.loc[full_df.State.isin(['Unspecified', 'Others']),'State'] = "UNKNOWN"

In [ ]:
full_df[full_df.Income_Range == "UNKNOWN"]

In [ ]:
full_df.Store_Format.value_counts()

In [ ]:
# dropping customer id and Store Code
# full_df.drop(['Customer_ID', 'Store_Code'], axis=1, inplace=True)

In [ ]:
cols = ['Gender', 'Income_Range', 'Job_Type', 'Language', 'Loyalty_Status',
       'Marital_Status', 'Nationality', 'State', 'Store_Format', 'Region_Code','Geo_Field' ]

In [ ]:
for i in cols:
    full_df[i] = full_df[i].astype('category')

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
full_df[cols] = full_df[cols].apply(le.fit_transform)

In [ ]:
full_df.head(2)

In [ ]:
full_df.columns

In [ ]:
# Splitting back to train set
train_df = full_df[:ntrain]
test_df = full_df[ntrain:]
y = train_df['Prediction']
X = train_df.drop('Prediction', axis=1)

In [ ]:
train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

In [ ]:
train_df.Prediction.value_counts()

In [ ]:
train_df.to_csv("train_df_wo.csv", index=False)

In [ ]:
test_df.columns

In [ ]:
len(train_df)

# BALANCE

In [1]:
import modin.pandas as pd
train_df = pd.read_csv("final_train.csv")

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2018-12-13_08-11-33_22395/logs.
Waiting for redis server at 127.0.0.1:25226 to respond...
Waiting for redis server at 127.0.0.1:14344 to respond...
Starting the Plasma object store with 6.579558809 GB memory using /dev/shm.


In [2]:
from imblearn.over_sampling import SMOTE

In [3]:
sm = SMOTE(random_state=2)

In [4]:
cols = train_df.drop("Prediction", axis=1).columns

In [5]:
cols = cols.append(pd.Index(["Prediction"]))

In [6]:
cols

Index(['Customer_ID', 'Store_Code', 'Nationality', 'Income_Range', 'Job_Type',
       'Marital_Status', 'Gender', 'State', 'Language', 'Loyalty_Status',
       'Points', 'First_txn_days', 'Last_accr_txn_days', 'Age', 'Week',
       'Invoices', 'Item_Count', 'Revenue', 'Discount', 'Last_Pur_Year',
       'Return_Reason', 'Region_Code', 'Store_Format', 'Sales_Per_Day',
       'Geo_Field', 'Store_Size_Sq_Ft', 'isMall', 'Prediction'],
      dtype='object')

In [7]:
X_train_res, y_train_res = sm.fit_sample(train_df.drop("Prediction", axis=1), train_df.Prediction.ravel())

In [8]:
train_sm_df = pd.DataFrame(X_train_res)

In [9]:
test_sm_df = pd.DataFrame(y_train_res)

In [10]:
X_train_res.shape

(5831426, 27)

In [11]:
train_sm_df = pd.concat([train_sm_df, test_sm_df], axis=1)

In [12]:
train_sm_df.columns = ['Customer_ID', 'Store_Code', 'Nationality', 'Income_Range', 'Job_Type',
       'Marital_Status', 'Gender', 'State', 'Language', 'Loyalty_Status',
       'Points', 'First_txn_days', 'Last_accr_txn_days', 'Age', 'Week',
       'Invoices', 'Item_Count', 'Revenue', 'Discount', 'Last_Pur_Year',
       'Return_Reason', 'Region_Code', 'Store_Format', 'Sales_Per_Day',
       'Geo_Field', 'Store_Size_Sq_Ft', 'isMall', 'Prediction']

In [13]:
train_sm_df.to_csv("balanced_set_new.csv", index=False)

/home/vivekviswambharan/anaconda3/envs/3point6/lib/python3.6/site-packages/modin/pandas/dataframe.py:3964: UserWarning: Defaulting to Pandas implementation
  warnings.warn("Defaulting to Pandas implementation", UserWarning)


# Prepare Validation Set

In [ ]:
train_df = pd.read_csv("train_df_wo.csv")

In [ ]:
train_df.Customer_ID.head(2)

In [ ]:
test_df = train_df[train_df.Store_Code.isin(['60075', '60250', '60077', '60079' ])]

In [ ]:
train_df.drop(train_df[train_df.Store_Code.isin(['60075', '60250', '60077', '60079' ])].index, axis=0, inplace=True)

In [ ]:
test_df.drop(['Customer_ID','Store_Code'],axis=1, inplace=True)
train_df.drop(['Customer_ID','Store_Code'],axis=1, inplace=True)

In [ ]:
y_train = train_df.Prediction
X_train = train_df.drop("Prediction", axis=1)
y_test = test_df.Prediction
X_test = test_df.drop("Prediction", axis=1)

In [ ]:
len(X_train)

# XGBOOST

# RANDOM FOREST

In [ ]:
 # removing unimportant features
X_train.drop(['Gender', 'Last_Pur_Year', 'Loyalty_Status', 'Marital_Status', 'Income_Range', 'Job_Type', 'Store_Format', 'Last_Pur_Week'],\
                             axis=1,inplace=True)
X_test.drop(['Gender', 'Last_Pur_Year', 'Loyalty_Status', 'Marital_Status', 'Income_Range', 'Job_Type', 'Store_Format', 'Last_Pur_Week'],\
                            axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
feature_names = X_train.columns 
# len(feature_names)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [20, 50, 100],
    'max_features': [2, 7],
    'min_samples_leaf': [3, 5, 10],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 5)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=feature_names).sort_values(ascending=False)
feature_imp

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import f1_score
# Model Accuracy, how often is the classifier correct?
# prediction on test set
# print("AUC - ROC of oob : ", roc_auc_score(y_train, clf.oob_score_))

y_pred=clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_pred, y_test))
conf_mat = confusion_matrix(y_test, y_pred)
print("conf_mat:", conf_mat)
auc_roc = roc_auc_score(y_test, y_pred)
print("auc_roc", auc_roc)
f1 = f1_score(y_test, y_pred)
print("f1:", f1)